In [ ]:
!pip install openai langchain pypdf tiktoken sentence_transformers chromadb

  Using cached openai-1.10.0-py3-none-any.whl (225 kB)
  Using cached langchain-0.1.5-py3-none-any.whl (806 kB)
  Using cached pypdf-4.0.1-py3-none-any.whl (283 kB)
  Using cached tiktoken-0.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = '8c91c5c7058847f3820f9d7c12bbb1fb'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://sktfly-ai.openai.azure.com/'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-05-15'

In [ ]:
# Data의 loading
import openai
import tiktoken

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

loader = PyPDFLoader('/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf')
pages = loader.load_and_split()

In [ ]:
pages = loader.load_and_split()

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
                  chunk_size=500,
                  chunk_overlap=50,
                  length_function=tiktoken_len
)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name='jhgan/ko-sbert-nli'
model_kwargs={'device':'cpu'}
encode_kwargs={'normalize_embeddings':True}

hf = HuggingFaceEmbeddings(
        model_name = model_name,
        model_kwargs = model_kwargs,
        encode_kwargs = encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# split
docs = text_splitter.split_documents(pages)

db = Chroma.from_documents(docs, hf)

In [ ]:
# 언어 모델의 생성
chatgpt = AzureChatOpenAI(
            deployment_name='dev-gpt-35-turbo'
)

In [ ]:
qa = RetrievalQA.from_chain_type(
                    llm = chatgpt,
                    chain_type = 'stuff',
                    retriever = db.as_retriever(
                                    search_type = 'mmr',
                                    search_kwargs = {'k':3,'fetch_k':10}
                    ),
                    return_source_documents=True
)

In [ ]:
query='탄소중립에 중소기업이 필요한 이유는?'

In [ ]:
result = qa(query)
result

{'query': '탄소중립에 중소기업이 필요한 이유는?',
 'result': '중소기업의 적극적인 참여가 필요한 이유는 탄소중립을 실현하기 위해 국가 경제에서 중소기업이 차지하는 비중이 높기 때문입니다. 중소기업의 참여를 통해 탄소중립에 대한 노력이 더욱 강화될 수 있고, 중소기업이 친환경 기술과 제품 개발, 사업 전환에 적극적으로 참여함으로써 경쟁력을 확보하고 성장할 수 있는 기회를 얻을 수 있습니다.',
 'source_documents': [Document(page_content='대한 대응 필요성을 느끼지 못한다는 결과를 보여주고 있음\n∙향후 중소기업을 대상으로 도입될 정부 정책과 제도에 대한 관심과 적극적 참여가 필요함 \n- 국가 경제에서 중소기업이 차지하는 비중이 높은 우리나라는 정부와 몇몇 대기업의 노력만으로 탄소중립\n을 실현하기 불가능함\n- 따라서 탄소중립에 대한 중소기업의 적극적 참여가 필요하며 , 중소기업의 참여를 유인할 수 있는 다양한 \n정부 정책과 제도가 마련될 예정임\n- 지원책을 적극 활용하여 많은 중소기업이 친환경 기술 및 제품 개발, 사업 전환 등에 적극적으로 참여한\n다면, 탄소중립 시대에서 경쟁할 수 있는 역량을 확보하고 성장할 수 있는 동력을 마련할 수 있을 것임', metadata={'page': 11, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향 (1).pdf'}),
  Document(page_content='속화 추진\n- 산업 분야에서는 재생에너지 , 수소, 에너지 IT 등 3대 에너지 신산업 육성에 집중\n- (고탄소 산업구조 혁신) 철강, 석유화학 등 탄소 다배출 업종에 대한 기술개발 지원, 고탄소 중소기업 대\n상 맞춤형 공정개선 지원 등을 통해 저탄소 산업구조로의 전환 추진\n- 금속･화학제품제조업 등 다배출 중소기업을 대상으로 공정진단을 통해 맞춤형 공정개선 ‧설비보급을 지원\n